In [1]:
import torch
import torch.nn as nn
import torchvision.models as models
import torch.nn.functional as F
import numpy as np

## 依照指示取出模型特定層的資訊

In [74]:
# Define model
class TheModelClass(nn.Module):
    def __init__(self):
        super(TheModelClass, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16* 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Initialize model
model = TheModelClass()


### 取出 self.pool層兩次的輸出，包含：
* x = self.pool(F.relu(self.conv1(x)))
* x = self.pool(F.relu(self.conv2(x)))

In [75]:
outputs= []
def layer1_hook(module, input_, output):
    outputs.append(output)

model.pool.register_forward_hook(layer1_hook)

In [76]:
input_ = torch.randn(1, 3, 32, 32)
output = model(input_)

In [77]:
outputs

[tensor([[[[0.3829, 0.1059, 0.9578,  ..., 0.7338, 0.5987, 0.6708],
           [0.0000, 1.0411, 0.5577,  ..., 0.4875, 0.6466, 0.1773],
           [0.3411, 0.8940, 0.2930,  ..., 0.0408, 0.0000, 0.4324],
           ...,
           [0.8634, 0.7097, 0.0000,  ..., 1.5065, 0.3688, 0.4858],
           [0.6978, 0.3997, 0.8356,  ..., 0.4410, 0.1473, 0.4136],
           [0.5336, 1.4814, 0.2338,  ..., 0.3680, 0.5213, 0.0000]],
 
          [[0.0587, 0.5024, 0.2446,  ..., 0.1523, 0.1762, 0.6358],
           [1.2661, 0.7968, 0.5191,  ..., 0.7436, 0.5459, 0.6484],
           [0.0000, 0.5319, 0.6775,  ..., 0.5055, 0.8672, 0.0000],
           ...,
           [0.7034, 0.7570, 0.4250,  ..., 0.6569, 0.9446, 0.5339],
           [0.7089, 1.3587, 0.5382,  ..., 0.6618, 0.0782, 0.3372],
           [0.7828, 0.6801, 0.5957,  ..., 0.9995, 0.5030, 0.3499]],
 
          [[0.0000, 0.5227, 0.7917,  ..., 1.3293, 0.6492, 0.1216],
           [0.9113, 1.0809, 0.7229,  ..., 0.2167, 0.6590, 0.6120],
           [1.7735, 0.00

In [78]:
print(outputs[0].shape)
print(outputs[1].shape)

torch.Size([1, 6, 14, 14])
torch.Size([1, 16, 5, 5])


## 加入自定義 initialization fuction

#### 對所有Conv2D層使用自定義initialization function
* weight : nn.init.kaiming_normal_
* bias : 全部輸入1

In [80]:
from torch.nn import init

def weights_init(m):
    #classname = m.__class__.__name__
    if isinstance(m, nn.Conv2d):
        #torch.nn.init.xavier_uniform_(m.weight)
        nn.init.kaiming_normal_(m.weight.data,
                                    a=0,
                                    mode='fan_out',
                                    nonlinearity='relu')
        if m.bias is not None:
            torch.nn.init.ones_(m.bias)
            
model.apply(weights_init)

TheModelClass(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

#### 查看 conv層的bias是否皆為1

In [81]:
for name, parameters in model.named_parameters():
    if ('conv' in name) and ('bias' in name):
        print(name, parameters)
        print('\n')

conv1.bias Parameter containing:
tensor([1., 1., 1., 1., 1., 1.], requires_grad=True)


conv2.bias Parameter containing:
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       requires_grad=True)


